In [1]:
import numpy as np
import pandas as pd
import time
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, execute, Aer
from qiskit.visualization import plot_histogram
from qiskit.aqua import Operator, run_algorithm

import random
from math import log, pi



dataTrain = pd.read_csv('Pancreas_Ovary_Train.txt', delimiter='\t')
dataTest = pd.read_csv('Pancreas_Ovary_Test.txt', delimiter='\t')

# transfer to np arrays and remove first columns
arrayTrain = np.array(dataTrain)[:, 1:]
arrayTest = np.array(dataTest)[:, 1:]

#Combining for now
array = np.vstack((arrayTrain, arrayTest))

In [2]:
# Use Aer's qasm_simulator
backend = Aer.get_backend('qasm_simulator')

In [3]:
y = array[:,:2]
data = array[:,2:]

In [50]:
def encode(circ, q, n, datapoint):
    for i in range(n):
        if datapoint[i] == 1:
            circ.x(q[i])
    return circ

def rotations(circ, q, n, params):
    for i in range(n):
        circ.u3(params[2*i], params[2*i+1], 0, q[i])

def FC(circ, q, n):
    for i in range(n):
        for j in range(n):
            if i != j:
                circ.cx(q[i], q[j])
                
def layer(circ, q, n, params):
    rotations(circ, q, n, params)
    FC(circ, q, n)
    return circ

def run_circuit(n, q, c, circ, params, datapoint):
    q = QuantumRegister(n)                  #can these be initialized outside the function?
    c = ClassicalRegister(n)
    circ = QuantumCircuit(q, c)
    
    num_params = len(params)
    num_layers = int(num_params/(2*n))
    
    circ = encode(circ, q, n, datapoint)
    
    for k in range(num_layers):
        circ = layer(circ, q, n, params[2*n*k : 2*n*(k+1)])
    
    #Measure 1st qubit (arbitrary) to be output
    circ.measure(q[0],c[0])
    
    result = execute(circ, backend = backend, shots = 1, memory = True)
    output = result.result().get_memory(circ)
    output = int(output[0][1])     #Turn result into either just integer 0 or 1
    
    return output

def costFunc(n, data, y, params):
    q = QuantumRegister(n)                  #can these be initialized outside the function?
    c = ClassicalRegister(n)
    circ = QuantumCircuit(q, c)
    
    y = y.astype('int')
    
    accuracy = 0
    for i, datapoint in enumerate(data):
        output = run_circuit(n, q, c, circ, params, datapoint)
        circ.reset(q)
        if output == y[i, 0]:
            accuracy += 1
    accuracy /= len(data)
    
    cost = -accuracy
    
    return cost

In [51]:
# Particle swarm optimizer

class Particle:
    def __init__(self,x0):
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual
        self.err_i=-1               # error individual

        for i in range(0,num_dimensions):
            self.velocity_i.append(random.uniform(-0.25,0.25))
            self.position_i.append(x0[i])

    # evaluate current fitness
    def evaluate(self, n, data, y, costFunc):
        self.err_i=costFunc(n, data, y, self.position_i)

        # check to see if the current position is an individual best
        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i

    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.75       # constant inertia weight (how much to weigh the previous velocity)
        c1=2        # cognative constant
        c2=2        # social constant
        
        for i in range(0,num_dimensions):
            r1=random.random()
            r2=random.random()
            
            # We need to account for the fact that parameters are angles
            dist_cog = self.pos_best_i[i]-self.position_i[i]
            if dist_cog > pi:
                dist_cog -= 2*pi
            if dist_cog < -pi:
                dist_cog += 2*pi
                
            dist_soc = pos_best_g[i] - self.position_i[i]
            if dist_soc > pi:
                dist_soc -= 2*pi
            if dist_soc < -pi:
                dist_soc += 2*pi

            vel_cognitive=c1*r1*dist_cog
            vel_social=c2*r2*dist_soc
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]

            #account for particles crossing the 0=2pi line
            if self.position_i[i] > 2*pi:
                self.position_i[i] -= 2*pi
            
            if self.position_i[i] < 0:
                self.position_i[i] += 2*pi
    
                
class PSO():
    def __init__(self,n, data, y,costFunc,x0,num_particles,maxiter):
        global num_dimensions

        num_dimensions=len(x0)
        self.err_best_g=-1                   # best error for group
        self.pos_best_g=[]                   # best position for group

        # establish the swarm
        swarm=[]
        for i in range(0,num_particles):
            swarm.append(Particle(x0))

        # begin optimization loop
        i=0
        while i < maxiter:
            # cycle through particles in swarm and evaluate fitness
            for j in range(0,num_particles):
                swarm[j].evaluate(n, data, y, costFunc)

                # determine if current particle is the best (globally)
                if swarm[j].err_i < self.err_best_g or self.err_best_g == -1:
                    self.pos_best_g=list(swarm[j].position_i)
                    self.err_best_g=float(swarm[j].err_i)

            # cycle through swarm and update velocities and position
            for j in range(0,num_particles):
                swarm[j].update_velocity(self.pos_best_g)
                swarm[j].update_position()
            i+=1
            
          #  if i%2 == 0:
            print("Iteration %d: %f" % (i, self.err_best_g))

        # print final results
        print('FINAL: %f' % self.err_best_g)
        
    
    def best(self):
        return self.pos_best_g
        

if __name__ == "__PSO__":
    main()

In [52]:
# Run optimization

n = len(data[0])

layers = 1
shots = 1000   #shots per run of given circuit; larger will give better statistics from runs

initial = np.random.uniform(0, 2*pi, size=2*n*layers)

best = PSO(n, data, y, costFunc, initial, num_particles=5, maxiter=4).best()

Iteration 1: -0.632653
Iteration 2: -0.632653


KeyboardInterrupt: 

In [45]:
from sklearn.svm import LinearSVC 

labels = y[:, 0]

labels = labels.astype('int')

svm = LinearSVC(C=1, loss="hinge")
svm.fit(data, labels)

predictions = svm.predict(data)
accuracy = sum(predictions == labels)/len(predictions)
print('Accuracy: %f' %accuracy)

Accuracy: 0.904762


In [44]:
type(labels)

numpy.ndarray

In [36]:
n = 2
q = QuantumRegister(n)                  #can these be initialized outside the function?
c = ClassicalRegister(n)
circ = QuantumCircuit(q, c)
    
circ.x(q[0])
circ.x(q[1])
circ.measure(q[0], c[0])
circ.measure(q[1], c[1])

circ.reset(q)
circ.measure(q[0], c[0])
circ.measure(q[1], c[1])


result = execute(circ, backend = backend, shots = 1, memory = True)
output = result.result().get_memory(circ)
output

['00']

In [ ]:
output

In [ ]:
initial = np.random.normal(size=2*n*layers)

In [ ]:
output = run_circuit(n, initial, data[0])
output

In [14]:
sum(y[:,0])

54